In [ ]:
import numpy as np
import pandas as pd
import fastFM
from fastFM.datasets import make_user_item_regression
from sklearn.model_selection import train_test_split
from fastFM import sgd
from fastFM import als
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from scipy.sparse import csc_matrix
from fastFM import mcmc
import functools as fct
import itertools as itools
import random, scipy

### Installation of fastFM
https://github.com/ibayer/fastFM

# Factorization Machine

There are two key advantages with this method:
- Linear complexity in learning the interaction between feature vectors
- Able to deal with sparse datasets

### Intuition:
Say we want to predict the rating $r_{u, i}$ (Rating for user $u$ and $i$) using SVDFunk, we would do:
\begin{equation}
    r_{u, i} = b_{u, i} + p^T_u q_i
\end{equation}

where $P \in \mathbb{R}^{|U| \times l}$ and $Q \in \mathbb{R}^{|I| \times l}$ are the learnt low-rank matrices with $l$ latent features and $b_{u, i}$ is the bias. Here we see that the feature (i.e., each component of the vectors $p_u$ and $q_i$) related to $u$ and $i$ are interacting with each other explicitly. FM models this interaction using a feature vector for each feature.

This means we can first have:
\begin{equation}
    \hat{y}(\textbf{x}) = w_0 + \sum^n_{i = 1} w_i x_i
\end{equation}
where $w_0$ is the bias, the $\textbf{x} \in \mathbb{R}^{1 \times n}$ is a feature vector for user $u$ to predict rating for item $i$ and $w_i$ is the weight assigned to each feature $x_i$. We note that this is the linear regression formula. But we have left out the interactions between features!

Therefore, we add the interactions between pairs of features as such:
\begin{equation}
    \hat{y}(\textbf{x}) = w_0 + \sum^n_{i = 1} w_i x_i + \sum^n_{i = 1} \sum^j_{j = i + 1} w_{i, j} x_i x_j
\end{equation}
for the interaction between the features $x_i$ and $x_j$, we assign a weight $w_{i, j}$. 

There's two problems with the above:
1. Under a sparse dataset, there's little observed interactions between features $x_i$ and $x_j$, so the weight $w_{i, j}$ is difficult to learn.
2. This model is expensive! It has a complexity of $\mathcal{O}(n^2)$. 

### Idea
Same as the idea of matrix factorization of the user-item matrix with high sparsity, we factorize the feature interaction weight matrix $\mathbf{W} \in \mathbb{R}^{n \times n}$ to $\mathbf{V} \in \mathbb{R}^{n \times k}$. Therefore, we are now estimating the weights for feature interactions: $\hat{w}_{i, j} = \langle \mathbb{v}_i, \mathbb{v}_j \rangle$. $\langle \cdot, \cdot \rangle$ is the dot product between two vectors. 

This gives the FM model equation:
\begin{equation}
    \hat{y}(\textbf{x}) = w_0 + \sum^n_{i = 1} w_i x_i + \sum^n_{i = 1} \sum^n_{j = i + 1} \langle \textbf{v}_i, \textbf{v}_j \rangle x_i x_j
\end{equation}
where the model parameters that have to be estimated are:
\begin{equation}
    w_0 \in \mathbb{R}, \textbf{w} \in \mathbb{R}^n, \textbf{V} \in \mathbb{R}^{n \times k}
\end{equation}
Once again $w_0$ is the bias, $\textbf{w}$ are the weights for each feature and $\textbf{V}$ are the vectors to estimate the weights to be assigned to the interactions between features. $k \in \mathbb{R}^+$ is a hyperparameter (a parameter to parametize other parameters) to define the dimensionality of the factorization. Obviously, we want $k < n$.

We need to show that we can in fact factorize the weight matrix $\mathbf{W}$. We know that we can since it is a positive definite matrix (because it's symmetrical and all the pivots are positive).

## However, what's so great about the FM model??
We see that the complexity is $\mathcal{O}(kn^2)$. Here comes the magic, we show that it can be computed in linear time $\mathcal{O}(kn)$. We first note that we only need to simplify the third term: 
\begin{equation}
    \sum^n_{i = 1} \sum^n_{j = i + 1} \langle \textbf{v}_i, \textbf{v}_j \rangle x_i x_j
\end{equation}

### Lemma 3.1 (clarified from FM paper)
\begin{equation}
    \sum^n_{i = 1} \sum^n_{j = i + 1} \langle \textbf{v}_i, \textbf{v}_j \rangle x_i x_j = \frac{1}{2} \sum^k_{f = 1}((\sum^n_{i = 1} v_{i, f} x_i)^2 - \sum^n_{i = 1} v^2_{i, f} x^2_i)
\end{equation}

Ignore $\langle \textbf{v}_i, \textbf{v}_j \rangle x_i x_j$, and think about a $n \times n$ square for $\sum^n_{i = 1} \sum^n_{j = 1}$ and that $\sum^n_{i = 1} \sum^n_{j = i + 1}$ is actually the upper or lower triangles. Upper and lower triangles are the same since dot product is commutative, i.e., $\langle \textbf{v}_i, \textbf{v}_j \rangle = \langle \textbf{v}_j, \textbf{v}_i \rangle$. Hence, we find that:

\begin{align}
    \sum^n_{i = 1} \sum^n_{j = 1} \langle \textbf{v}_i, \textbf{v}_j \rangle x_i x_j &= 2 \times \sum^n_{i = 1} \sum^n_{j = i + 1} \langle \textbf{v}_i, \textbf{v}_j \rangle x_i x_j + \sum^n_{i = 1} \langle \textbf{v}_i, \textbf{v}_i \rangle x_i x_i \\
    \sum^n_{i = 1} \sum^n_{j = i + 1} \langle \textbf{v}_i, \textbf{v}_j \rangle x_i x_j &= \frac{1}{2} (\sum^n_{i = 1} \sum^n_{j = 1} \langle \textbf{v}_i, \textbf{v}_j \rangle x_i x_j - \sum^n_{i = 1} \langle \textbf{v}_i, \textbf{v}_i \rangle x_i x_i) \\
    &= \frac{1}{2}(\sum^n_{i = 1} \sum^n_{j = 1} \sum^k_{f = 1} v_{i, f} v_{j, f} x_i x_j - \sum^n_{i = 1} \sum^k_{f = 1} v_{i, f} v_{j, f} x_i x_i) \\
    &= \frac{1}{2} \sum^k_{f = 1} ((\sum^n_{i = 1} v_{i, f} x_i) (\sum^n_{j = 1} v_{j, f} x_j) - \sum^n_{i = 1} v^2_{i, f} x^2_i) \\
    &= \frac{1}{2} \sum^k_{f = 1} ((\sum^n_{i = 1} v_{i, f} x_i)^2 - \sum^n_{i = 1} v^2_{i, f} x^2_i)
\end{align}

Using the above simplification, we see the model is linear in both $k$ and $n$. Hence the model complexity is $\mathcal{O}(kn)$. 


### Learning the parameters
I won't go through the details of how the parameters are learnt here, but they are learnt through gradient descent methods to minimize loss (difference between the estimate and ground truth value). Methods include Stochastic Gradient Descent (SGD), Alternating Least Square (ALS) and even Markov Chain Monte Carlo (MCMC).

### Intuition of usage
By modelling feature interaction through factorization, FM can mimic many of the existing factorization models. The key to the usage of FM is two things:
#### - Everything is a feature. Even the userId and itemId!
#### - It is all about using the right input data, i.e., the feature vector $\textbf{x}$

I show two examples to exemplify how existing factorization models can be translated to a FM equivalent. 
### SVDFunk/ Matrix factorization
The original model:
\begin{equation}
    r_{u, i} = b_{u, i} + p^T_u q_i
\end{equation}

The FM equivalent:
\begin{equation}
    \hat{y}(\textbf{x}) = w_0 + w_u + w_i + \langle \textbf{v}_u, \textbf{v}_i \rangle
\end{equation}
The feature vector $\textbf{x}$ has $n = |U \cup I|$ components and for each component $x_j$, $x_j = 1$ if $j = i$ or $j = u$, otherwise $x_j = 0$. This means that the feature vector marks a 1 for the feature that we are currently interested in, i.e., the user $u$ and the item $i$ to be rated. 

$w_0$ is the overall bias. $w_u$ and $w_i$ comes from the first component $\sum^n_{j = 1} w_j x_j$. Since all $x_j = 0$ if $j \neq u$ nor $j \neq i$, the summation is just left with $w_u x_u$ and $w_i x_i$ and since $x_u = x_i = 1$, we are just left with $w_u$ and $w_i$. The same idea applies to the third component $\sum^n_{i = 1} \sum^n_{j = i + 1} \langle \textbf{v}_i, \textbf{v}_j \rangle x_i x_j$ to get $\langle \textbf{v}_u, \textbf{v}_i \rangle$, the iteraction between $x_u$ and $x_i$. 

### SVD++
The original model:
\begin{equation}
    r_{u, i} = \mu + b_u + b_i + (p_u + \frac{1}{\sqrt{|N(u)|}} \sum_{j \in N(u)} y_j)^T q_i
\end{equation}
where $N(u)$ is the set of item that the user $u$ has rated. We see that we need to add additional interactions between $\frac{1}{\sqrt{|N(u)|}} \sum_{j \in N(u)} y_j$ (rated items) and $q_i$ (item to be rated). 

The FM equivalent:
\begin{equation}
    \hat{y}(\textbf{x}) = w_0 + w_u + w_i + \langle \textbf{v}_u, \textbf{v}_i \rangle + \frac{1}{\sqrt{|N(u)|}} \sum_{l \in N(u)} \langle \textbf{v}_i, \textbf{v}_l \rangle + \frac{1}{\sqrt{|N(u)|}} \sum_{l \in N(u)} (w_l + \langle \textbf{v}_u, \textbf{v}_l \rangle + \frac{1}{\sqrt{|N(u)|}} \sum_{l^{'} \in N(u), l^{'} > l} \langle v_l, v^{'}_l \rangle)
\end{equation}

We see that the first five terms are related to the feature interactions of the SVD++ model, i.e., the FM equivalent is mimicking the SVD++ model. The last term is an addition of the FM equivalent to model the interaction between users and items $N(u)$, $\langle v_u, v_l \rangle$, as well as basic effects for each item $N(u)$, $w_l$ and the interactions between pairs of items $N(u)$, $\frac{1}{\sqrt{|N(u)|}} \sum_{l^{'} \in N(u), l^{'} > l} \langle v_l, v^{'}_l \rangle$.

Next we showcase FM prediction using the fastFM library.

## Matrix factorization using MovieLens 100K

In [ ]:
# import movie lens 
movies_dir = './datasets/u.item'
ratings_dir = './datasets/u.data'

movies_col = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies_df = pd.read_csv(movies_dir,  sep='|', names=movies_col, usecols=range(5), encoding='latin-1')
ratings_col = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_df = pd.read_csv(ratings_dir, index_col=None, sep='\t', names=ratings_col)

In [ ]:
movies_df.head()

In [ ]:
ratings_df.head()

In [ ]:
# number of movies and ratings
num_of_movies = len(movies_df['movie_id'].unique().tolist())
num_of_users = len(ratings_df['user_id'].unique().tolist())
num_of_ratings = ratings_df.shape[0]

print('Num. of movies: {}\nNum. of users: {}\nNum. of ratings: {}'.format(num_of_movies, num_of_users, num_of_ratings))

In [ ]:
# distribution of number of ratings per movie
movies_df['numOfRatings'] = movies_df['movie_id'].apply(lambda movie_id: ratings_df[(ratings_df['movie_id']==movie_id)].shape[0])

In [ ]:
# sort by number of ratings
movies_df.sort_values('numOfRatings', ascending=True, inplace=True)

In [ ]:
# index
index = np.arange(num_of_movies)
width = 0.35

# histogram
freq_plot = movies_df['numOfRatings'].hist(figsize=(20, 8), color='orange', bins=30)
freq_plot.set_ylabel('Number of ratings', fontsize=25)
freq_plot.set_xlabel('Number of movies', fontsize=25)
freq_plot.set_title('Frequency distribution of ratings per movie', fontsize=25)
freq_plot.set_xlim([0, 250])

plt.show()

### Useful method to convert data into FM input format

In [ ]:
def get_single_entries_in_fm_input_format(data, itemlist):
    '''Create the needed input (data, (row, column)) format for csc matrix for
    the single entries in data. Each entry would take one row. This means it
    would result in a csc matrix with dimension (|data| x |itemlist|).

    :param data: single entry data
    :type data: narray
    :param itemlist: ordered list of possible item values
    :type itemlist: narray

    :return: data, row indexes, column indexes, shape
    :rtype: tuple
    '''
    column = len(itemlist)
    row = len(data)
    shape = (row, column)

    row_inds = np.zeros(len(data), dtype=np.int)
    col_inds = np.zeros(len(data), dtype=np.int)
    datalist = np.zeros(len(data), dtype=np.float)
    for i in range(len(data)):
        item = data[i]
        val = 1
        datalist[i] = val
        # locate its position in the itemlist, throws error if item is not a
        # possible item
        col_ind = np.where(itemlist==item)[0]
        # should not be duplicated items in the itemlist
        assert len(col_ind) == 1
        col_ind = col_ind[0]
        row_ind = i

        col_inds[i] = col_ind
        row_inds[i] = row_ind

    return datalist, row_inds, col_inds, shape


def get_multi_entries_in_fm_input_format(data, itemlist, norm_func=None):
    '''Create the needed input (data, (row, column)) format for csc matrix for
    the multi entries in data. Each set of multi entries would take one row.
    This means it would result in a csc matrix with dimension
    (|entry sets in data| x |itemlist|).

    :param data: multi entry sets in data
    :type data: a multidimension narray
    :param itemlist: ordered list of possible item values
    :type itemlist: narray
    :param norm_func: normalization function
    :type norm_func: function that receives the size of each multi entry

    :return: datalist, row indexes, column indexes, shape
    :rtype: tuple
    '''
    column = len(itemlist)
    # number of sets of entries in data
    row = len(data)
    shape = (row, column)

    # number of data
    num_of_data = fct.reduce(lambda x, y: x + len(y), data, 0)
    row_inds = np.zeros(num_of_data, dtype=np.int)
    col_inds = np.zeros(num_of_data, dtype=np.int)
    datalist = np.zeros(num_of_data, dtype=np.float)
    cnt = 0
    for i in range(len(data)):
        multi_entry = data[i]

        if norm_func != None:
            # function that receives the size of the multi_entry to decide how to normalize it
            val = norm_func(len(multi_entry))
        else:
            # default binary value assignment
            val = 1 if len(multi_entry) > 0 else 0

        # for each entry in multi_entry, locate its position in the itemlist,
        # throws error if item is not a possible item
        # all the entries stay at the same row
        row_ind = i
        for item in multi_entry:
            col_ind = np.where(itemlist==item)[0]
            assert len(col_ind) == 1
            col_ind = col_ind[0]
            
            datalist[cnt] = val
            col_inds[cnt] = col_ind
            row_inds[cnt] = row_ind
            
            # update count
            cnt += 1

    return datalist, row_inds, col_inds, shape

In [ ]:
# convert the movielens dataset into FM input format
movielist = movies_df.sort_values('movie_id')['movie_id'].values
userlist = ratings_df.sort_values('user_id')['user_id'].unique()

# user who did ratings
user_data = ratings_df['user_id'].values
# movies rated
movie_data = ratings_df['movie_id'].values
# target vector: ratings
rating_data = ratings_df['rating'].values

# convert to FM input format
user_datalist, user_row_inds, user_col_inds, user_shape = get_single_entries_in_fm_input_format(data=user_data, 
                                                                                                itemlist=userlist)
movie_datalist, movie_row_inds, movie_col_inds, movie_shape = get_single_entries_in_fm_input_format(data=movie_data,
                                                                                                   itemlist=movielist)

# concat the two columns by shifting column indexes of columns related to movies
# shift by the number of columns in user columns
shift_by = len(userlist)
movie_col_inds += shift_by
# concat them
datalist = np.append(user_datalist, movie_datalist)
row_inds = np.append(user_row_inds, movie_row_inds)
col_inds = np.append(user_col_inds, movie_col_inds)
# make sure both feature set have the same number of rows
print('User feature set shape: {}\nMovie feature set shape: {}'.format(user_shape, movie_shape))
assert user_shape[0] == movie_shape[0]
shape = (user_shape[0], user_shape[1] + movie_shape[1])
print('Dimension of FM input: {}'.format(shape))

X = csc_matrix((datalist, (row_inds, col_inds)), shape=shape)
y = rating_data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

### Change rank = 10, 50, 100, 200

In [ ]:
%%time
fm = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=10, l2_reg_w=0.1, l2_reg_V=0.5)
fm.fit(X_train, y_train)
y_pred = fm.predict(X_test)

In [ ]:
error_als = mean_squared_error(y_test, y_pred)
print('Mean squared error under ALS: {}'.format(error_als))

### Change rank = 10, 50, 100, 200

In [ ]:
%%time
fm_sgd = sgd.FMRegression(n_iter=10000000, init_stdev=0.01, rank=100, random_state=123, 
                              l2_reg_w=0.1, l2_reg_V=0.5, step_size=0.01)
fm_sgd.fit(X_train, y_train)
y_pred_sgd = fm_sgd.predict(X_test)

In [ ]:
error_sgd = mean_squared_error(y_test, y_pred_sgd)
print('Mean squared error under SGD: {}'.format(error_sgd))

In [ ]:
%%time
seed = 123
rank = 8
# step size is the number of iterations to continue using the current coefficients (number of samples without taking a new step)
step_size = 10

fm_mcmc = mcmc.FMRegression(n_iter=0, init_stdev=0.1, rank=rank, random_state=seed)
# initialize the model and hyperparameters
fm_mcmc.fit_predict(X_train=X_train, y_train=y_train, X_test=X_test)

n_iter = 100
mse_test = np.zeros(n_iter - 1)
hyper_param = np.zeros((n_iter - 1, 3 + 2 * rank), dtype=np.float)
for nr, i in enumerate(range(1, n_iter)):
    fm_mcmc.random_state = i * seed
    y_pred = fm_mcmc.fit_predict(X_train, y_train, X_test, n_more_iter=step_size)
    mse_test[i - 1] = mean_squared_error(y_pred, y_test)
    hyper_param[nr,:] = fm_mcmc.hyper_param_


In [ ]:
values = np.arange(1, n_iter)
x = values * step_size
burn_in = 50
x = x[burn_in:]

fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, figsize=(20, 8))

axes[0, 0].plot(x, mse_test[burn_in:], label='test rmse', color="r")
axes[0, 0].legend(fontsize=15)
axes[0, 0].tick_params(labelsize=15)
axes[0, 1].plot(x, hyper_param[burn_in:,0], label='alpha', color="b")
axes[0, 1].legend(fontsize=15)
axes[0, 1].tick_params(labelsize=15)
axes[1, 0].plot(x, hyper_param[burn_in:,1], label='lambda_w', color="g")
axes[1, 0].legend(fontsize=15)
axes[1, 0].tick_params(labelsize=15)
axes[1, 1].plot(x, hyper_param[burn_in:,3], label='mu_w', color="g")
axes[1, 1].legend(fontsize=15)
axes[1, 1].tick_params(labelsize=15)

plt.show()